In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel("TR结果en.xlsx")

In [3]:
import re
# 合并df1、df2和df3的值到一个列表中
mylist1 = list(df1.values.flatten())
# 使用正则表达式提取所有单引号内的内容
pattern = re.compile(r"'(.*?)'")
content_set = set()
for item in mylist1:
    content_set.update(pattern.findall(item))
# 将集合转换为列表
content_list = list(content_set)
# 打印结果
len(content_list)

In [4]:
content = pd.DataFrame(content_list, columns=['words'])

In [5]:
content[df1.columns] = None

In [6]:
# 遍历每一列
for j, column in enumerate(df1.columns):
    # 获取当前列的数据
    column_data = df1[column]
    # 遍历每一个元素
    for i, value in enumerate(column_data):
        pattern = re.compile(r"'(.*?)'")
        element_list = pattern.findall(value)
        element = ''.join(element_list)
        pattern2 = re.compile(r', (.*?)\)')
        element_list2 = pattern2.findall(value)
        element2 = float(element_list2[0])
        # 检查元素是否存在于 content 中
        if element in content['words'].tolist():
            # 如果存在，则将值追加到对应的列
            content.loc[element, df1.columns[j]] = element2

In [7]:
content.to_excel("content.xlsx")

In [8]:
df2 = content.drop(content.index[0:len(content_list)])
df2 = df2.drop(columns=['words'])
df2

In [9]:
from scipy.spatial import distance
import pandas as pd

# 将缺失值设置为0
df2_filled = df2.fillna(0)

# 计算两两之间的余弦相似度
similarity_matrix = pd.DataFrame(columns=df2_filled.columns, index=df2_filled.columns)
for col1 in df2_filled.columns:
    for col2 in df2_filled.columns:
        if col1 != col2:
            vec1 = df2_filled[col1].tolist()
            vec2 = df2_filled[col2].tolist()
            similarity = 1 - distance.cosine(vec1, vec2)
            similarity_matrix.at[col1, col2] = similarity

In [1]:
similarities_1_2 = similarity_matrix.loc[['A0'], ['B0', 'B1']].reset_index(drop=True)
similarities_2_3 = similarity_matrix.loc[['B0', 'B1'], ['C0', 'C1', 'C2']].reset_index(drop=True)
similarities_3_4 = similarity_matrix.loc[['C0', 'C1', 'C2'], ['D0', 'D1', 'D2']].reset_index(drop=True)

# 设置阈值

In [3]:
import numpy as np
def find_similar_elements(matrix):
    matrix = matrix.to_numpy()
    rows, cols = np.where(matrix > 0)# set Threshold 
    values = matrix[rows, cols]
    similar_matrix = np.column_stack((rows, cols, values))
    return similar_matrix

In [4]:
similar_elements_1_2 = find_similar_elements(similarities_1_2)

In [5]:
import pandas as pd
similar_elements_1_2_pd = pd.DataFrame(similar_elements_1_2, columns=['Source', 'Target', 'Value'])
similar_elements_1_2_pd['Source'] = similar_elements_1_2_pd['Source'].apply(lambda x: 'A' + str(int(x)))
similar_elements_1_2_pd['Target'] = similar_elements_1_2_pd['Target'].apply(lambda x: 'B' + str(int(x)))

In [6]:
similar_elements_1_2_pd

In [7]:
similar_elements_2_3 = find_similar_elements(similarities_2_3)

In [8]:
import pandas as pd
similar_elements_2_3_pd = pd.DataFrame(similar_elements_2_3, columns=['Source', 'Target', 'Value'])
similar_elements_2_3_pd['Source'] = similar_elements_2_3_pd['Source'].apply(lambda x: 'B' + str(int(x)))
similar_elements_2_3_pd['Target'] = similar_elements_2_3_pd['Target'].apply(lambda x: 'C' + str(int(x)))
similar_elements_2_3_pd

In [9]:
similar_elements_3_4 = find_similar_elements(similarities_3_4)
# 创建新的 DataFrame

similar_elements_3_4_pd = pd.DataFrame(similar_elements_3_4, columns=['Source', 'Target', 'Value'])
similar_elements_3_4_pd['Source'] = similar_elements_3_4_pd['Source'].apply(lambda x: 'C' + str(int(x)))
similar_elements_3_4_pd['Target'] = similar_elements_3_4_pd['Target'].apply(lambda x: 'D' + str(int(x)))

In [10]:
similar_elements_1_2_3_pd = pd.concat([similar_elements_1_2_pd, similar_elements_2_3_pd, similar_elements_3_4_pd])

In [8]:
import pandas as pd

nodes = pd.Series(similar_elements_1_2_3_pd['Source'].tolist() + similar_elements_1_2_3_pd['Target'].tolist()).unique()

node_index = {node: index for index, node in enumerate(nodes)}

similar_elements_1_2_3_pd['source'] = similar_elements_1_2_3_pd['Source'].apply(lambda x: node_index[x])
similar_elements_1_2_3_pd['target'] = similar_elements_1_2_3_pd['Target'].apply(lambda x: node_index[x])

links = similar_elements_1_2_3_pd[['source', 'target', 'Value']]

In [9]:
from pyecharts.charts import Sankey
from pyecharts import options as opts
from pyecharts.globals import ThemeType
data = similar_elements_1_2_3_pd.values.tolist()

nodes = []
links = []

for item in data:
    source = item[0]
    target = item[1]
    value = item[2]

    if source not in nodes:
        nodes.append(source)
    if target not in nodes:
        nodes.append(target)

    links.append({"source": source, "target": target, "value": value})

sankey = (
    Sankey(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, bg_color="#ffffff",width="2000px", height="800px"))
    .add(
        series_name="",
        nodes=[{"name": node} for node in nodes],
        links=links,
        linestyle_opt=opts.LineStyleOpts(opacity=0.5, curve=0.3, color="source"),
        #itemstyle_opts=opts.ItemStyleOpts(border_width=1, border_color="#aaa")
    )
    .set_global_opts(title_opts=opts.TitleOpts(title=""),
                     #theme="wonderland" 
                     )
     
)
sankey.render("sankey.html")